# Informationen für dieses Notebook

Dieses .ipnynb dient dazu, den Datensatz "df_movie"_for_streamlit.csv" (befindet sich im Ordner 'Datasets') so vorzubereiten, dass er danach im .ipynb "data_encoding_scaling_similarity_matrix" genutzt werden kann um encoded und gescaled zu werden um eine similarity-matrix zu erstellen.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_theme(style="darkgrid", palette="bright", context="paper")

# Maximale Anzahl an Spalten und Breite festlegen
pd.set_option('display.max_columns', None)  # Zeigt alle Spalten an
pd.set_option('display.width', 1000)  # Breite des Outputs in Zeichen

In [2]:
# CSV-Datei aus dem Ordner Datasets laden
df_movie_suggestion = pd.read_csv('../../Datasets/df_movie_for_streamlit.csv')

# DataFrame anzeigen 
df_movie_suggestion.head(5)

,id,Title,Movie Link,Year,Duration,MPA,Rating,Votes,budget,grossWorldWide,gross_US_Canada,opening_weekend_Gross,directors,writers,stars,genres,countries_origin,filming_locations,production_companies,Languages,wins,nominations,oscars
0,tt0073195,Jaws,https://www.imdb.com/title/tt0073195,1975,124.0,PG,8.1,683000.0,7000000.0,477220580.0,266567580.0,7061513.0,['Steven Spielberg'],"['Peter Benchley', 'Carl Gottlieb']","['Roy Scheider', 'Robert Shaw', 'Richard Dreyf...","['Monster Horror', 'Sea Adventure', 'Survival'...",['United States'],"[""Water Street, Edgartown, Martha's Vineyard, ...","['Zanuck/Brown Productions', 'Universal Pictur...",['English'],0,20,0
1,tt0073629,The Rocky Horror Picture Show,https://www.imdb.com/title/tt0073629,1975,100.0,R,7.4,173000.0,1200000.0,115798478.0,112892319.0,NaN,['Jim Sharman'],"[""Richard O'Brien"", 'Jim Sharman']","['Tim Curry', 'Susan Sarandon', 'Barry Bostwick']","['Dark Comedy', 'Raunchy Comedy', 'Rock Musica...","['United Kingdom', 'United States']","['Oakley Court, Windsor Road, Oakley Green, Wi...","['Twentieth Century Fox', 'Michael White Produ...",['English'],0,4,0
2,tt0073486,One Flew Over the Cuckoo's Nest,https://www.imdb.com/title/tt0073486,1975,133.0,R,8.7,1.0,3000000.0,109115366.0,108981275.0,NaN,['Milos Forman'],"['Lawrence Hauben', 'Bo Goldman', 'Ken Kesey']","['Jack Nicholson', 'Louise Fletcher', 'Michael...","['Medical Drama', 'Psychological Drama', 'Drama']",['United States'],['Oregon State Mental Hospital - 2600 Center S...,"['Fantasy Films', 'N.V. Zvaluw']",['English'],0,15,0
3,tt0072890,Dog Day Afternoon,https://www.imdb.com/title/tt0072890,1975,125.0,R,8.0,279000.0,1800000.0,50002721.0,50000000.0,NaN,['Sidney Lumet'],"['Frank Pierson', 'P.F. Kluge', 'Thomas Moore']","['Al Pacino', 'John Cazale', 'Penelope Allen']","['Heist', 'True Crime', 'Biography', 'Crime', ...",['United States'],"['285 Prospect Park West, Brooklyn, New York C...","['Warner Bros.', 'Artists Entertainment Complex']",['English'],0,20,0
4,tt0073692,Shampoo,https://www.imdb.com/title/tt0073692,1975,110.0,R,6.4,15000.0,4000000.0,49407734.0,49407734.0,NaN,['Hal Ashby'],"['Robert Towne', 'Warren Beatty']","['Warren Beatty', 'Julie Christie', 'Goldie Ha...","['Satire', 'Comedy', 'Drama']",['United States'],"['2270 Bowmont Drive, Beverly Hills, Californi...","['Persky-Bright / Vista', 'Columbia Pictures',...",['English'],0,11,0


### Übersicht über Mende der Fehlende WErten je Spalte und Vorschläge damit umzugehen

- Berechnungen dafür wurden in einem anderen Notebook gemacht, hier nicht aufgeführt

Löschen?

| **Spalte**               | **Fehlende Werte (%)** | **Löschen oder behalten?** | **Begründung** |
|--------------------------|---------------------|------------------------|----------------|
| `budget`                | **65% fehlen** | **Löschen** | Zu viele fehlende Werte, schwer zu ersetzen |
| `grossWorldWide`        | **45% fehlen** | **Löschen** | Gilt auch für `gross_US_Canada` |
| `gross_US_Canada`       | **48% fehlen** | **Löschen** | Lokale Einnahmen oft unklar |
| `opening_weekend_Gross` | **54% fehlen** | **Löschen** | Viele Filme haben keine Start-Wochenendzahlen |
| `filming_locations`     | **20% fehlen** | **Löschen** | Keine Relevanz für Empfehlungen |

Behalten?

| **Spalte**          | **Fehlende Werte (%)** | **NaN-Behandlung** |
|---------------------|---------------------|-----------------|
| `MPA`              | **24% fehlen** | `"Unknown"` setzen |
| `Duration`         | **0.7% fehlen** | Median setzen |
| `Rating`           | **0.4% fehlen** | Median pro Jahr oder Genre setzen |
| `Votes`            | **0.4% fehlen** | `0` setzen (keine Votes = kein Rating) |
| `directors`        | **1% fehlen** | `"Unknown"` setzen |
| `writers`          | **5% fehlen** | `"Unknown"` setzen |
| `stars`            | **1.5% fehlen** | `"Unknown"` setzen |
| `genres`           | **1.1% fehlen** | Leere Liste `[]` setzen |
| `countries_origin` | **1.0% fehlen** | Leere Liste `[]` setzen |
| `Languages`        | **1.4% fehlen** | Leere Liste `[]` setzen |




In [3]:
df_movie_suggestion.drop(columns=['id', 'Movie Link', 'budget', 'grossWorldWide', 'gross_US_Canada', 'opening_weekend_Gross', 'filming_locations', 'Votes'], inplace=True)  # Mehrere Spalten entfernen

### Check für NaNs und Nullwerte

In [ ]:
# Zählen von fehlenden Werten (NaNs) und leeren Strings in jeder Spalte
missing_values = df_movie_suggestion.isnull().sum()  # Zählt NaNs
empty_strings = (df_movie_suggestion == "").sum()   # Zählt leere Strings

# Kombiniere die Ergebnisse in einem DataFrame zur besseren Übersicht
missing_summary = pd.DataFrame({'NaNs': missing_values, 'Empty Strings': empty_strings})
missing_summary['Total Missing'] = missing_summary['NaNs'] + missing_summary['Empty Strings']

# Zeige die Spalten an, die mindestens eine fehlende oder leere Zelle haben
missing_summary = missing_summary[missing_summary['Total Missing'] > 0]

missing_summary

,NaNs,Empty Strings,Total Missing
Duration,221,0,221
MPA,7976,0,7976
Rating,138,0,138
directors,359,0,359
writers,1576,0,1576
stars,473,0,473
genres,382,0,382
countries_origin,366,0,366
production_companies,1378,0,1378
Languages,474,0,474


In [5]:
# Liste aller Spalten außer MPA
columns_to_check = [col for col in df_movie_suggestion.columns if col != 'MPA']

# Direktes Überschreiben des Original-DataFrames mit bereinigten Daten
df_movie_suggestion = df_movie_suggestion.dropna(subset=columns_to_check)

# Ergebnis überprüfen
print(f"🔍 Verbleibende Filme: {len(df_movie_suggestion)} von ursprünglich 33600")

🔍 Verbleibende Filme: 30867 von ursprünglich 33600


### Überkategorien für MPA bilden

In [6]:
# Mapping der MPA-Kategorien
mpa_categories = {
    "All Ages (+0)": ['G', 'TV-G', 'TV-Y', 'TV-Y7', 'K-A', 'Approved'],
    "Parental Guidance (+13)": ['PG', 'PG-13', 'M/PG', 'TV-PG', 'TV-13', 'TV-14'],
    "Mature Audiences (+18)": ['R', 'MA-17', 'TV-MA', 'NC-17', 'X'],
    "Not Rated/Other": ['Not Rated', 'Unrated', 'Passed', 'GP']
}

# Mapping umkehren für schnelles Zuordnen
mpa_mapping = {}
for category, labels in mpa_categories.items():
    for label in labels:
        mpa_mapping[label] = category

# Spalte in allgemeine Kategorien umwandeln
df_movie_suggestion['MPA_category'] = df_movie_suggestion['MPA'].map(mpa_mapping)

# Fehlende Werte in `MPA_category` mit "Unknown" auffüllen
df_movie_suggestion['MPA_category'].fillna("Unknown", inplace=True)

# Alte `MPA`-Spalte entfernen
df_movie_suggestion.drop(columns=['MPA'], inplace=True)

C:\Users\diver\AppData\Local\Temp\ipykernel_24648\890967096.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_movie_suggestion['MPA_category'].fillna("Unknown", inplace=True)


In [7]:
# Spaltennamen in Kleinbuchstaben umwandeln
df_movie_suggestion.columns = df_movie_suggestion.columns.str.lower()

In [8]:
import ast  # Um Strings korrekt zu parsen

# Sicherer Parsing-Prozess für Strings
def safe_parse(value):
    try:
        # Nur Strings parsen
        if isinstance(value, str):
            return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return []  # Fehlerhafte Werte als leere Liste setzen
    return value  # Rückgabe des Originalwertes, wenn kein String

# Liste der Spalten, die Listen enthalten sollten
list_columns = ['directors', 'writers', 'stars', 'genres', 'countries_origin', 'languages', 'production_companies']

# Safe Parsing auf alle Listen-Spalten anwenden
for col in list_columns:
    df_movie_suggestion[col] = df_movie_suggestion[col].apply(safe_parse)

In [9]:
# Listen-Spalten, in denen NaN-Werte durch leere Listen ersetzt werden sollen
list_cols = ['genres', 'countries_origin', 'languages', 'production_companies', 'directors', 'writers', 'stars']

# NaN-Werte durch leere Listen ersetzen
for col in list_cols:
    df_movie_suggestion[col] = df_movie_suggestion[col].apply(lambda x: x if isinstance(x, list) else [])

In [10]:
# Einzigartige Subgenres extrahieren
unique_subgenres = sorted({subgenre for genres in df_movie_suggestion['genres'] for subgenre in genres})

# Ausgabe der einzigartigen Subgenres
unique_subgenres

['Action',
 'Action Epic',
 'Adult Animation',
 'Adventure',
 'Adventure Epic',
 'Alien Invasion',
 'Animal Adventure',
 'Animation',
 'Anime',
 'Artificial Intelligence',
 'B-Action',
 'B-Horror',
 'Baseball',
 'Basketball',
 'Biography',
 'Body Horror',
 'Body Swap Comedy',
 'Boxing',
 'Buddy Comedy',
 'Buddy Cop',
 'Bumbling Detective',
 'Caper',
 'Car Action',
 'Classic Musical',
 'Classical Western',
 'Comedy',
 'Coming-of-Age',
 'Computer Animation',
 'Concert',
 'Conspiracy Thriller',
 'Contemporary Western',
 'Cop Drama',
 'Costume Drama',
 'Cozy Mystery',
 'Crime',
 'Crime Documentary',
 'Cyber Thriller',
 'Cyberpunk',
 'Dark Comedy',
 'Dark Fantasy',
 'Dark Romance',
 'Desert Adventure',
 'Dinosaur Adventure',
 'Disaster',
 'Docudrama',
 'Documentary',
 'Drama',
 'Drug Crime',
 'Dystopian Sci-Fi',
 'Epic',
 'Erotic Thriller',
 'Extreme Sport',
 'Fairy Tale',
 'Faith & Spirituality Documentary',
 'Family',
 'Fantasy',
 'Fantasy Epic',
 'Farce',
 'Feel-Good Romance',
 'Financia

In [11]:
genre_categories = {
    "Action": [
        'Action', 'Action Epic', 'B-Action', 'Car Action', 'Gun Fu', 
        'Martial Arts', 'One-Person Army Action', 'Kung Fu', 
        'Samurai', 'Sword & Sandal', 'Swashbuckler', 'Buddy Cop', 
        'Spy'
    ],
    "Abenteuer": [
        'Adventure', 'Adventure Epic', 'Animal Adventure', 'Desert Adventure', 
        'Dinosaur Adventure', 'Globetrotting Adventure', 'Jungle Adventure', 
        'Mountain Adventure', 'Sea Adventure', 'Space Sci-Fi', 'Urban Adventure', 
        'Quest', 'Survival', 'Road Trip'
    ],
    "Horror": [
        'Horror', 'B-Horror', 'Monster Horror', 'Psychological Horror', 
        'Slasher Horror', 'Supernatural Horror', 'Teen Horror', 
        'Vampire Horror', 'Werewolf Horror', 'Zombie Horror', 
        'Splatter Horror', 'Found Footage Horror', 'Folk Horror', 
        'Giallo', 'Kaiju', 'Witch Horror', 'Body Horror'
    ],
    "Thriller": [
        'Thriller', 'Conspiracy Thriller', 'Political Thriller', 'Erotic Thriller', 
        'Psychological Thriller', 'Serial Killer', 'Heist', 'Crime', 'Cop Drama', 
        'Drug Crime', 'Gangster', 'Police Procedural', 'Legal Thriller', 
        'Suspense Mystery'
    ],
    "Science-Fiction": [
        'Sci-Fi', 'Sci-Fi Epic', 'Dystopian Sci-Fi', 'Cyber Thriller', 
        'Cyberpunk', 'Steampunk', 'Alien Invasion', 'Artificial Intelligence', 
        'Time Travel', 'Mecha', 'Superhero'
    ],
    "Fantasy": [
        'Fantasy', 'Dark Fantasy', 'Fantasy Epic', 'Sword & Sorcery', 
        'Supernatural Fantasy', 'Fairy Tale', 'Isekai', 'Slice of Life', 
        'Teen Fantasy'
    ],
    "Komödie": [
        'Comedy', 'Dark Comedy', 'Farce', 'High-Concept Comedy', 'Quirky Comedy', 
        'Raunchy Comedy', 'Romantic Comedy', 'Screwball Comedy', 'Slapstick', 
        'Stoner Comedy', 'Teen Comedy', 'Buddy Comedy', 'Parody', 'Satire', 
        'Sketch Comedy', 'Mockumentary', 'Sitcom'
    ],
    "Drama": [
        'Drama', 'Costume Drama', 'Cop Drama', 'Crime', 'Docudrama', 'Family', 
        'Historical Epic', 'Legal Drama', 'Medical Drama', 'Period Drama', 
        'Political Drama', 'Prison Drama', 'Psychological Drama', 'Showbiz Drama', 
        'Teen Drama', 'Tragedy', 'Workplace Drama', 'Financial Drama', 
        'Korean Drama', 'Biography', 'Dark Romance'
    ],
    "Romantik": [
        'Romance', 'Feel-Good Romance', 'Holiday Romance', 'Teen Romance', 
        'Tragic Romance', 'Steamy Romance', 'Romantic Epic'
    ],
    "Musik und Tanz": [
        'Musical', 'Classic Musical', 'Pop Musical', 'Rock Musical', 
        'Concert', 'Music', 'Jukebox Musical', 'Music Documentary'
    ],
    "Sport": [
        'Baseball', 'Basketball', 'Boxing', 'Extreme Sport', 'Football', 
        'Motorsport', 'Soccer', 'Sport', 'Sports Documentary', 'Water Sport'
    ],
    "Krimi": [
        'True Crime', 'Heist', 'Drug Crime', 'Gangster', 'Hard-boiled Detective', 
        'Bumbling Detective', 'Cozy Mystery'
    ],
    "Dokumentationen": [
        'Documentary', 'Crime Documentary', 'Food Documentary', 
        'Military Documentary', 'Nature Documentary', 
        'Political Documentary', 'Science & Technology Documentary', 
        'Travel Documentary', 'Faith & Spirituality Documentary', 'History Documentary', 
        'Game Show', 'News', 'Reality TV', 'Talk Show'
    ],
    "Western": [
        'Western', 'Contemporary Western', 'Classical Western', 
        'Spaghetti Western', 'Western Epic'
    ],
    "Animation": [
        'Animation', 'Anime', 'Hand-Drawn Animation', 'Stop Motion Animation', 
        'Adult Animation', 'Computer Animation'
    ],
    "Kinder und Familie": [
        'Family', 'Holiday Family', 'Teen Adventure', 'Coming-of-Age', 
        'Holiday', 'Holiday Animation', 'Holiday Comedy'
    ],
    "Historisch": [
        'Historical Epic', 'History', 'Period Drama'
    ],
    "Krieg": [
        'War', 'War Epic'
    ],
    "Mystery": [
        'Mystery', 'Whodunnit'
    ],
    "Sonstiges": [
        'Epic', 'Stand-Up', 'Iyashikei', 'Josei', 'Seinen', 'Shōjo', 
        'Shōnen', 'Soap Opera'
    ]
}



In [12]:
# Funktion, um Subgenres zu den zugehörigen Überkategorien zuzuordnen
def map_genres_to_categories(genres, mapping):
    categories = set()  # Verwende ein Set, um Duplikate automatisch zu vermeiden
    for genre in genres:
        for category, subgenres in mapping.items():
            if genre in subgenres:
                categories.add(category)
    return list(categories)  # Set zurück in eine Liste umwandeln

# Neue `genres`-Spalte mit Überkategorien erstellen
df_movie_suggestion['genres'] = df_movie_suggestion['genres'].apply(
    lambda x: map_genres_to_categories(x, genre_categories)
)

In [13]:
# Median für `Duration` setzen
df_movie_suggestion['duration'].fillna(df_movie_suggestion['duration'].median(), inplace=True)

# `Rating` und `Votes`: Realistische Platzhalter setzen
df_movie_suggestion['rating'].fillna(-1, inplace=True)  # -1 für "Unknown Rating"

C:\Users\diver\AppData\Local\Temp\ipykernel_24648\1022458875.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_movie_suggestion['duration'].fillna(df_movie_suggestion['duration'].median(), inplace=True)
C:\Users\diver\AppData\Local\Temp\ipykernel_24648\1022458875.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

In [14]:
# Zählen von fehlenden Werten (NaNs) und leeren Strings in jeder Spalte
missing_values = df_movie_suggestion.isnull().sum()  # Zählt NaNs
empty_strings = (df_movie_suggestion == "").sum()   # Zählt leere Strings

# Kombiniere die Ergebnisse in einem DataFrame zur besseren Übersicht
missing_summary = pd.DataFrame({'NaNs': missing_values, 'Empty Strings': empty_strings})
missing_summary['Total Missing'] = missing_summary['NaNs'] + missing_summary['Empty Strings']

# Zeige die Spalten an, die mindestens eine fehlende oder leere Zelle haben
missing_summary = missing_summary[missing_summary['Total Missing'] > 0]

missing_summary

,NaNs,Empty Strings,Total Missing


In [15]:
# DataFrame als CSV speichern
df_movie_suggestion.to_csv('cleaned_movie_suggestion.csv', index=False)

print("Die Datei wurde erfolgreich als 'cleaned_movie_suggestion.csv' gespeichert!")

Die Datei wurde erfolgreich als 'cleaned_movie_suggestion.csv' gespeichert!


### Weiter gehts im .ipynb "02_data_encoding_scaling_similarity_matrix"